# Get BLAST outputs

In [4]:
import os

In [5]:
write_root = "/labs/mignot/KIR_BLAST_OUT"
read_root = "/labs/mignot/10xGenomics/5k_pbmc_v3_fastqs"
blastdb_path = "/home/ashteng/blastdb/KIR.fasta"

In [6]:
os.path.basename(os.listdir(read_root)[0])

'5k_pbmc_v3_S1_L003_R1_001.fastq.gz'

In [7]:
for filepath in os.listdir(read_root):
    print(filepath)
    basename = filepath.split(".")[0]
    save_path = os.path.join(write_root, basename)
    with open("{}.sh".format(basename), "w") as f:
        f.write("#!/bin/bash\n#SBATCH --job-name=default\n#SBATCH --nodes=1\n#SBATCH --ntasks=1\n#SBATCH --cpus-per-task=8\n#SBATCH --partition=batch\n#SBATCH --account=mignot\n#SBATCH --time=10:00:00\n")    
        #command1 = "gunzip < {} > {}.fastq\n".format(os.path.join(read_root, filepath), os.path.join(write_root, save_path))
        #f.write(command1)
        #command2 = "paste - - - - < {}.fastq | cut -f 1,2 | sed 's/^@/>/' | tr '\t' '\n' > {}.fasta\n".format(save_path, save_path)
        #f.write(command2)
    #     command3 = "rm {}.fastq\n".format(save_path)
    #     f.write(command3)
        command4 = "blastn -num_threads 8 -db {} -query {}.fasta -outfmt 6 -out {}.out\n".format(blastdb_path, save_path, save_path)
        f.write(command4)
#         command5 = "rm {}.fasta\n".format(save_path)
#         f.write(command5)

5k_pbmc_v3_S1_L003_R1_001.fastq.gz
5k_pbmc_v3_S1_L002_R2_001.fastq.gz
5k_pbmc_v3_S1_L001_R2_001.fastq.gz
5k_pbmc_v3_S1_L002_I1_001.fastq.gz
5k_pbmc_v3_S1_L003_I1_001.fastq.gz
5k_pbmc_v3_S1_L004_I1_001.fastq.gz
5k_pbmc_v3_S1_L004_R2_001.fastq.gz
5k_pbmc_v3_S1_L002_R1_001.fastq.gz
5k_pbmc_v3_S1_L001_I1_001.fastq.gz
5k_pbmc_v3_S1_L001_R1_001.fastq.gz
5k_pbmc_v3_S1_L004_R1_001.fastq.gz
5k_pbmc_v3_S1_L003_R2_001.fastq.gz


In [8]:
for filepath in os.listdir("."):
    if filepath[-3:] == ".sh":
        basename, _ = filepath.split(".")
        script = "{}.sh".format(os.path.join(basename))
        print(script)
        !sbatch {script}

5k_pbmc_v3_S1_L004_R1_001.sh
Submitted batch job 12663388
5k_pbmc_v3_S1_L003_R2_001.sh
Submitted batch job 12663389
5k_pbmc_v3_S1_L003_R1_001.sh
Submitted batch job 12663390
5k_pbmc_v3_S1_L004_R2_001.sh
Submitted batch job 12663391
5k_pbmc_v3_S1_L004_I1_001.sh
Submitted batch job 12663392
5k_pbmc_v3_S1_L003_I1_001.sh
Submitted batch job 12663393
5k_pbmc_v3_S1_L001_R1_001.sh
Submitted batch job 12663394
5k_pbmc_v3_S1_L001_R2_001.sh
Submitted batch job 12663395
5k_pbmc_v3_S1_L002_R2_001.sh
Submitted batch job 12663396
5k_pbmc_v3_S1_L002_I1_001.sh
Submitted batch job 12663397
5k_pbmc_v3_S1_L001_I1_001.sh
Submitted batch job 12663398
5k_pbmc_v3_S1_L002_R1_001.sh
Submitted batch job 12663399


# Generate Dataframe

In [45]:
import pandas as pd
import numpy as np

In [46]:
root = "/labs/mignot/KIR_BLAST_OUT"

In [ ]:
df = pd.DataFrame(columns=["patid", "genotype", "dose", "transcript", "count"])
for file in os.listdir(root):
    patid, extension = file[:-4], file[-4:]
    if extension == ".out":
        filepath = os.path.join(root, file)
        # load each person's .out file into a separate dataframe
        raw_df = pd.read_csv(filepath, delimiter="\t", names=['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore'])
        # group by read id and only extract the maximum score read across families
        # raw_df = raw_df.loc[raw_df.groupby(["qseqid"], sort=False)["pident"].idxmax()] 
        for j_code in all_j_codes_dict:
            df = df.append({"patid": patid, "genotype": genotype, "dose": genotype_name_dose_dict[genotype], "transcript": j_code, "count": all_j_codes_dict[j_code]}, ignore_index=True)